say it with me: who is great?

In [8]:
import pygame
import random
import numpy as np
from collections import deque
import torch
import torch.nn as nn
import torch.optim as optim
from dinosaur import Dinosaur  # Assuming this contains the Dinosaur class with image rendering
from obstacle import Obstacle  # Assuming this contains the Obstacle class with image rendering
from batsymbol import Batsymb  # Assuming this contains BatSymbol for power-ups or extra obstacles

# Screen dimensions
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 400

# Game constants
VELOCITY = 5

# Initialize Pygame
pygame.init()

# Screen setup
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
clock = pygame.time.Clock()

# DQN model definition
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim),
        )

    def forward(self, x):
        return self.fc(x)

# Get the game state
def get_game_state(dinosaur, obstacles, velocity):
    closest_obstacle = min(obstacles, key=lambda obs: obs.get_distance_to(dinosaur))
    dino_rect = dinosaur.get_rect()
    state = [
        dino_rect.x,  # Dinosaur's x-position
        dino_rect.y,  # Dinosaur's y-position
        closest_obstacle.get_distance_to(dinosaur),  # Distance to the closest obstacle
        velocity,  # Current velocity
    ]
    return state

# Initialize models and training components
state_dim = 4  # dino_x, dino_y, obstacle_distance, velocity
action_dim = 2  # Actions: [Do nothing, Jump]
model = DQN(state_dim, action_dim)
target_model = DQN(state_dim, action_dim)
target_model.load_state_dict(model.state_dict())

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Replay memory
memory = deque(maxlen=2000)
gamma = 0.99
epsilon = 1.0
epsilon_decay = 0.995
epsilon_min = 0.01


In [ ]:
# Training function
def train_dqn():
    if len(memory) < 32:
        return
    batch = random.sample(memory, 32)
    states, actions, rewards, next_states, dones = zip(*batch)

    states = torch.tensor(states, dtype=torch.float32)
    actions = torch.tensor(actions, dtype=torch.int64)
    rewards = torch.tensor(rewards, dtype=torch.float32)
    next_states = torch.tensor(next_states, dtype=torch.float32)
    dones = torch.tensor(dones, dtype=torch.float32)

    q_values = model(states).gather(1, actions.unsqueeze(1)).squeeze(1)
    next_q_values = target_model(next_states).max(1)[0]
    target_q_values = rewards + (1 - dones) * gamma * next_q_values

    loss = criterion(q_values, target_q_values)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Game loop and training integration
for episode in range(500):  # Number of episodes
    dinosaur = Dinosaur()  # Instantiate Dinosaur from your file
    obstacles = [Obstacle()]  # Instantiate Obstacles
    total_reward = 0
    running = True

    while running:
        screen.fill((255, 255, 255))  # White background

        # Game events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                pygame.quit()

        # Get current state
        state = get_game_state(dinosaur, obstacles, VELOCITY)

        # Epsilon-greedy action selection
        if np.random.rand() < epsilon:
            action = random.choice([0, 1])  # Random action
        else:
            with torch.no_grad():
                state_tensor = torch.tensor(state, dtype=torch.float32)
                q_values = model(state_tensor)
                action = torch.argmax(q_values).item()

        # Perform action
        if action == 1:  # Jump
            dinosaur.jump()

        # Update game entities
        dinosaur.update()
        for obstacle in obstacles:
            obstacle.update()
            if obstacle.is_off_screen():
                obstacles.remove(obstacle)
                obstacles.append(Obstacle())  # Add a new obstacle

        # Check for collisions
        reward = 1  # Reward for survival
        if any(obstacle.collides_with(dinosaur) for obstacle in obstacles):
            reward = -10  # Penalty for collision
            running = False

        total_reward += reward

        # Update state
        next_state = get_game_state(dinosaur, obstacles, VELOCITY)
        done = not running

        # Store transition in memory
        memory.append((state, action, reward, next_state, done))

        # Train the model
        train_dqn()

        # Render game objects
        dinosaur.draw(screen)
        for obstacle in obstacles:
            obstacle.draw(screen)

        pygame.display.flip()
        clock.tick(30)

    # Update epsilon and target network
    epsilon = max(epsilon * epsilon_decay, epsilon_min)
    if episode % 10 == 0:
        target_model.load_state_dict(model.state_dict())


TypeError: Dinosaur.__init__() missing 1 required positional argument: 'surfaceHeight'

In [10]:
import pygame
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
from pygame import mixer
from dinosaur import Dinosaur
from obstacle import Obstacle
from batsymbol import Batsymb

# Initialize pygame and set up game window
pygame.init()
size = width, height = 640, 480
gameDisplay = pygame.display.set_mode(size)
clock = pygame.time.Clock()

# Initialize background music
mixer.init()
mixer.music.load("bgm.mp3")
mixer.music.set_volume(0.7)
mixer.music.play(loops=-1)

# Initialize game variables
ground_height = height - 100
game_timer = 0
collision = False
dinosaur = Dinosaur(ground_height)
last_frame = pygame.time.get_ticks()
MINGAP = 200
MAXGAP = 600
MAXSIZE = 40
MINSIZE = 20
obstacles = []
last_obstacle = width
ground_image = pygame.image.load(r"ground.png")
ground_width = ground_image.get_width()
ground_scroll = 0

# Initialize DQN parameters
ACTION_SPACE = 3  # [0: duck, 1: jump, 2: do nothing]
STATE_SIZE = 4  # Example state size, you can increase the state size based on the game
BATCH_SIZE = 32
GAMMA = 0.99
LEARNING_RATE = 0.001
EPSILON = 1.0
EPSILON_DECAY = 0.995
EPSILON_MIN = 0.01
TRAINING_START = 1000
MEMORY_SIZE = 2000

# Experience replay buffer
memory = deque(maxlen=MEMORY_SIZE)

# Define the neural network for DQN
class DQN(nn.Module):
    def __init__(self):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(STATE_SIZE, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, ACTION_SPACE)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the agent and the model
model = DQN()
target_model = DQN()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Copy weights from model to target_model
target_model.load_state_dict(model.state_dict())
target_model.eval()

# Define function to select action based on epsilon-greedy policy
def select_action(state):
    if random.random() < EPSILON:
        return random.randint(0, ACTION_SPACE - 1)
    state = torch.FloatTensor(state).unsqueeze(0)
    q_values = model(state)
    return torch.argmax(q_values).item()

# Define function to store experience in the replay buffer
def store_experience(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

# Define function to train the DQN
def train_dqn():
    if len(memory) < BATCH_SIZE:
        return

    batch = random.sample(memory, BATCH_SIZE)
    states, actions, rewards, next_states, dones = zip(*batch)

    states = torch.FloatTensor(states)
    next_states = torch.FloatTensor(next_states)
    actions = torch.LongTensor(actions)
    rewards = torch.FloatTensor(rewards)
    dones = torch.FloatTensor(dones)

    # Get Q-values for current states and actions
    q_values = model(states).gather(1, actions.unsqueeze(1)).squeeze(1)

    # Get Q-values for next states from target model
    next_q_values = target_model(next_states).max(1)[0]
    target_q_values = rewards + (GAMMA * next_q_values * (1 - dones))

    # Calculate loss and optimize
    loss = nn.MSELoss()(q_values, target_q_values)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Update epsilon for exploration
    global EPSILON
    EPSILON = max(EPSILON * EPSILON_DECAY, EPSILON_MIN)

# Game loop
while True:
    t = pygame.time.get_ticks()
    delta_time = (t - last_frame) / 1000.0
    last_frame = t

    # Get the current state of the game
    state = np.array([dinosaur.x, dinosaur.y, len(obstacles), ground_scroll])

    # Select action using the DQN
    action = select_action(state)

    # Perform action
    if action == 0:  # Duck
        dinosaur.duck(True)
    elif action == 1:  # Jump
        dinosaur.bigjump()
    else:  # Do nothing
        dinosaur.duck(False)

    # Simulate the game and get reward
    game_display.fill((255, 255, 255))  # White background
    ground_scroll -= VELOCITY * delta_time
    if ground_scroll <= -ground_width:
        ground_scroll += ground_width
    game_display.blit(ground_image, (ground_scroll, 300))

    draw_text(f"Score: {int(game_timer * 10)}", text_font, (0, 255, 0), 100, 50)
    dinosaur.update(delta_time)
    dinosaur.draw(game_display)

    # Add new obstacles
    if len(obstacles) == 0 or obstacles[-1].x < width - MINGAP:
        ahaha = random.random()
        is_high = ahaha > 0.7
        obstacle_size = random.randint(MINSIZE, MAXSIZE) if not is_high else 30
        obstacles.append(Obstacle(last_obstacle, obstacle_size, ground_height, is_high))
        last_obstacle += MINGAP + (MAXGAP - MINGAP) * random.random() + 0.01 * game_timer * 1000

    # Handle obstacle movement and collision
    reward = -1  # Default reward for each step (penalty)
    for obs in obstacles:
        obs.update(delta_time, VELOCITY)
        obs.draw(game_display)

        dino_rect = pygame.Rect(dinosaur.x, dinosaur.surfaceHeight - dinosaur.y - dinosaur.height, dinosaur.width, dinosaur.height)
        obs_rect = pygame.Rect(obs.x, obs.y, obs.size, obs.size)

        if dino_rect.colliderect(obs_rect):
            reward = -10  # Large penalty for collision
            done = True
            break
    else:
        reward = 1  # Small reward for surviving
        done = False

    # Next state after taking action
    next_state = np.array([dinosaur.x, dinosaur.y, len(obstacles), ground_scroll])

    # Store experience in memory buffer
    store_experience(state, action, reward, next_state, done)

    # Train the DQN
    train_dqn()

    # Update the target model every few steps
    if t % 100 == 0:
        target_model.load_state_dict(model.state_dict())

    pygame.display.update()
    clock.tick(60)  # Limit the frame rate


NameError: name 'game_display' is not defined